In [3]:
from util import load_eidc_data
from sentence_transformers import SentenceTransformer

df = load_eidc_data.load_as_df('data/catalogue_metadata.json')

/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(df['description'])

/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
from scipy import spatial
import numpy as np


def get_top_n_datasets(query_embedding, datasets, dataset_embeddings, n):
    similarity_scores = []
    for doc_embedding in dataset_embeddings:
        dist = 1 - spatial.distance.cosine(query_embedding, doc_embedding)
        similarity_scores.append(dist)
    sorted = np.array(similarity_scores).argsort()[::-1][:n]
    return datasets.iloc[sorted]

In [10]:
questions = ['Where is the wettest soil in the UK?',
             'Where is water quality worst in the UK?',
             'Where are bird populations declining in the UK?',
             'Where in the UK are bumblebees most at risk from neonicotinoids?',
             'Which county in the UK has the most rivers?']

for q in questions:
    print(q)
    embedding = embedding_model.encode(q)
    result = get_top_n_datasets(embedding, df, embeddings, 3)
    print(result.to_markdown())

Where is the wettest soil in the UK?
157     Topsoil physico-chemical properties from the U...
236     Topsoil physico-chemical properties from the U...
1294    Topsoil physico-chemical properties from the U...
Name: title, dtype: object
Where is water quality worst in the UK?
290     Water quality data from the Ribble and Wyre ca...
305     Weekly water quality data from the River Thame...
1802    Weekly water quality data from the River Thame...
Name: title, dtype: object
Where are bird populations declining in the UK?
184     Diet, timing of egg laying and breeding succes...
1772    The Isle of May long-term study (IMLOTS) seabi...
1824    The Isle of May long-term study (IMLOTS) seabi...
Name: title, dtype: object
Where in the UK are bumblebees most at risk from neonicotinoids?
341     Location data of worker bumblebees across an a...
1755    Population responses of honeybees to oilseed r...
1436    The number and individual weight of bumblebees...
Name: title, dtype: object
Which 